In [15]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [33]:
#### All of the next functions are related to prox of g
#### (Projection onto dual flow space) ######
def f_forward(C,alpha):
    n = C.shape[0]
    return C.sum(axis=1).reshape(n,1)+C.sum(axis = 0).reshape(1,n) +(alpha)*C

def f_backward(C,alpha):
    #print C.shape
    n = C.shape[0]
    pt = C.sum(axis=1).reshape(n,1)
    a = -1.0/((n+alpha)*(alpha))
    c = -2*a/(2*n+alpha)
    
    p = pt*a
    q = C.sum(axis=0).reshape(1,n)*a
    k = pt.sum()*c
    
    return (C/(alpha) + p + q + k)

def big_inverse(C,n):
    # n is the number of dual sample points!!!
    
    C0 = f_forward(C,1)
    # inverse
    C0 = f_backward(C0,1+n/2.0)
    return f_forward(C-C0,1)/n

In [128]:
n=5
k = 3.9
# precomputation of matrix for diagonal cancelation       
V = np.zeros((n,n))
t = []
for i in range(n):
    if i == 0:
        V[i,i] = 1
    else:
        V[i-1,i-1] = 0
        V[i,i] = 1
    t.append(np.diag(big_inverse(V,k)))

M = np.stack(t)
c1 = M[0,1]
c2 = M[0,0] - c1

# routine for normalizing diagonal for projection
def normalize_diag(V,k):
    d = np.diag(big_inverse(V,k))
    l_norm = (d - c1*d.sum()/(c2+n*c1))/c2
    return V-np.diag(l_norm)

In [138]:
beta = 1+.5*k
c1new =  ((-(1-beta)**2)/beta)*(2/((n+beta)*(2*n+beta)))/k
c2new =  (-(1-beta)/beta)*(1-2*(1-beta)/(n+beta))/k

In [142]:
c1-c1new

-3.4694469519536142e-18

In [143]:
c2-c2new

0.0

In [111]:
d

-0.34416850386784786

In [112]:
M = np.zeros((n,n))
M[0,0] = 1.0
V = f_forward(M - f_backward(f_forward(M,1),beta),1)
V

array([[ 0.80027753,  0.06745094,  0.06745094,  0.06745094,  0.06745094,
         0.06745094,  0.06745094,  0.06745094,  0.06745094,  0.06745094,
         0.06745094,  0.06745094,  0.06745094,  0.06745094,  0.06745094],
       [ 0.06745094, -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ,
        -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ,
        -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ],
       [ 0.06745094, -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ,
        -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ,
        -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ],
       [ 0.06745094, -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ,
        -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ,
        -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ],
       [ 0.06745094, -0.0043587 , -0.0043587 , -0.0043587 , -0.0043587 ,
        -0.0043587 , -0.0043587 , -0.0043587 , 

In [113]:
a = -(1-beta)/beta
b = (beta-1)*(2*n+beta-1)
c = (n+beta)*(2*n+beta)
a*(1+2*b/c)

0.8002775270504336

In [115]:
a = (1-beta)**2
b = (beta*(n+beta))
c = 2/(2*n+beta)
(1-c)*a/b

0.06745093718215549

In [116]:
a = (1-beta)**2/beta
b = (n+beta)*(2*n+beta)
-a*(2/b)

-0.0043587035335803225

In [73]:
a = (1-beta)**2

array([[ 0.43296703, -0.07472527, -0.07472527, -0.07472527],
       [-0.07472527,  0.01758242,  0.01758242,  0.01758242],
       [-0.07472527,  0.01758242,  0.01758242,  0.01758242],
       [-0.07472527,  0.01758242,  0.01758242,  0.01758242]])

In [95]:
(1/beta)*(1/(n+beta))*((1-beta))+c

-0.07472527472527472

In [82]:
-(1/beta)*(1/(n+beta))+c

-0.04395604395604395

In [85]:
((n+1)/(n+beta))/beta

-0.29010989010989013

In [88]:
e1 = np.zeros((n,1))
e1[0] =1
o1 = np.ones((n,1))
o1t = np.ones((1,n))

# 

In [97]:
X = (o1.dot(e1.transpose())+e1.dot(o1t))
-((1-beta)/(beta))*( M- X/(n+beta)+2/((2*n+beta)*(n+beta)) )

array([[ 0.43296703, -0.07472527, -0.07472527, -0.07472527],
       [-0.07472527,  0.01758242,  0.01758242,  0.01758242],
       [-0.07472527,  0.01758242,  0.01758242,  0.01758242],
       [-0.07472527,  0.01758242,  0.01758242,  0.01758242]])

In [94]:
((n+1)*(o1.dot(e1.transpose())+e1.dot(o1t)))/(beta+n+beta)-c

array([[ 1.09352869,  0.53797314,  0.53797314,  0.53797314],
       [ 0.53797314, -0.01758242, -0.01758242, -0.01758242],
       [ 0.53797314, -0.01758242, -0.01758242, -0.01758242],
       [ 0.53797314, -0.01758242, -0.01758242, -0.01758242]])